<a href="https://colab.research.google.com/github/jeffersonramelo/workshop-ufu/blob/main/Fog_Index_copomPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas requests PyPDF2

In [39]:
pip install PyPDF2 --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
import pandas as pd
import requests
from urllib.parse import quote
import PyPDF2
import re


In [33]:
import tempfile

In [22]:
# Lendo o arquivo e transformando em um DataFrame
url = "https://www.bcb.gov.br/api/servico/sitebcb/atascopom/ultimas?quantidade=1000&filtro="
response = requests.get(url)
data = response.json()
df = pd.DataFrame(data["conteudo"])

In [23]:
# Excluindo linhas NA da coluna 'Url'
df = df.dropna(subset=['Url'])

In [24]:
# Isolando apenas os números das atas a partir da coluna 'Titulo'
df['ata'] = df['Titulo'].str.extract('(^\\d{2,3})').astype(float)

In [25]:
# Concatenando o endereço do Banco Central com a coluna 'Url' para baixar os PDFs
df['link'] = 'https://www.bcb.gov.br' + df['Url'].apply(quote)

In [43]:
# Função para extrair texto de um PDF
def extract_text_from_pdf(url):
    response = requests.get(url, stream=True)
    
    with tempfile.NamedTemporaryFile() as temp_file:
        with open(temp_file.name, 'wb') as f:
            f.write(response.content)
        
        pdf_reader = PyPDF2.PdfReader(temp_file.name)
        
        if not pdf_reader.is_encrypted and len(pdf_reader.pages) > 0:
            text = ''
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text += page.extract_text()
            return text
        else:
            return ''


In [ ]:
# Aplicando a função de extração de texto aos links dos PDFs e adicionando a coluna 'texto' ao DataFrame
df['texto'] = df['link'].apply(extract_text_from_pdf)

In [ ]:
# Visualizando o DataFrame final
print(df)


Salvar

In [49]:
import os

nome_arquivo = 'atascopomPT.csv'
caminho_arquivo = os.path.join(os.getcwd(), nome_arquivo)

df.to_csv(caminho_arquivo, index=False)


In [50]:
import pandas as pd
import math

# Função para calcular o Fog Index
def calculate_fog_index(text):
    words = text.split()
    num_sentences = text.count('.') + text.count('!') + text.count('?')
    num_words = len(words)
    num_complex_words = 0

    for word in words:
        if len(word) > 3 and len(word) >= 3 and word[-1] in ['s', 'd'] and word[-2] not in ['s', 'd']:
            num_complex_words += 1

    average_words_sentence = num_words / num_sentences
    percentage_complex_words = (num_complex_words / num_words) * 100
    fog_index = 0.4 * (average_words_sentence + percentage_complex_words)

    return fog_index

# Carregar o DataFrame a partir do arquivo CSV
df = pd.read_csv('atascopomPT.csv')

# Aplicar a função calculate_fog_index à coluna 'texto' e criar a coluna 'fog_index'
df['fog_index'] = df['texto'].apply(calculate_fog_index)

# Exibir o DataFrame com a coluna 'fog_index'
print(df)


          DataReferencia                                         ImagemCapa  \
0   2023-05-03T03:00:00Z  /content/publicacoes/PublishingImages/Capas/at...   
1   2023-03-22T03:00:00Z  /content/publicacoes/PublishingImages/Capas/at...   
2   2023-02-01T03:00:00Z  /content/publicacoes/PublishingImages/Capas/at...   
3   2022-12-07T03:00:00Z  /content/publicacoes/PublishingImages/Capas/at...   
4   2022-10-26T03:00:00Z  /content/publicacoes/PublishingImages/Capas/at...   
5   2022-09-21T03:00:00Z  /content/publicacoes/PublishingImages/Capas/at...   
6   2022-08-03T03:00:00Z  /content/publicacoes/PublishingImages/Capas/at...   
7   2022-06-15T03:00:00Z  /content/publicacoes/PublishingImages/Capas/at...   
8   2022-05-04T03:00:00Z  /content/publicacoes/PublishingImages/Capas/at...   
9   2022-03-16T03:00:00Z  /content/publicacoes/PublishingImages/Capas/at...   
10  2022-02-02T03:00:00Z  /content/publicacoes/PublishingImages/Capas/at...   
11  2021-12-08T03:00:00Z  /content/publicacoes/Publi